**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Introducción al aprendizaje reforzado

## Temporal difference

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Función de valor](#Función-de-valor)
* [Q-Learning](#Q-Learning)
* [El código](#El-código)


## Referencias

1. Adaptado de Markel Sanz, [Introducción al aprendizaje por refuerzo](https://medium.com/@markelsanz14/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-2-q-learning-883cd42fb48e)
2. Sutton, R. S., & Barto, A. G. (2018). [Reinforcement learning: An introduction. MIT press](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf).

## Introducción

<figure>
<center>
<img src="../Imagenes/Reinforcement_learning_diagram.png" width="400" height="300" align="center"/>
</figure>

Fuente: [Wikipedia](https://es.wikipedia.org/wiki/Aprendizaje_por_refuerzo)

## Ambiente no determinístico    

En el caso de que el entorno no sea determinista, tanto la recompensa como la acción son
probabilísticas. 

El nuevo sistema es un Markov decisión process(MDP) estocástico. 

Para reflejar lo no determinista
recompensa, la nueva función de valor es:

$$
V^{\pi}(S_t) = \mathbb{E}[R_t] = \mathbb{E}\left[\sum_{k=0}^{T} \lambda^k r_{t+k}\right]
$$


La ecuación de Bellman modificada es 

$$
Q(s,a) =  \mathbb{E}_{S'}\left[r + \max_{a'} Q(s',a')\right]
$$

En esta lección tratremos con ambientes determinísticos.

## Aprendizaje diferencia temporal


Q-learning es un caso especial de un tipo de aprendizaje más 
general llamado aprendizaje temporal diferencial.

Este tipo de aprendizaje se basa en la ecuación

$$
\large
Q(s,a) = Q(s,a) + \alpha(r + \lambda \max_{a^{'}}  Q(s^{'},a^{'})- Q(s,a)),
$$

La cual es similar a la ecuación de Bellman. A vece esta ecuación es llamada *generalized Q-learning*.

## Sarsa y bootstrapping

En los ejemplos anteriores no hemos usado ningń tipo de política. Un ejemplo del uso de una política de un paso es el algortimo SARSA que es definido de manera similar como

$$
\large
Q(s,a) = Q(s,a) + \alpha(r + \lambda  Q(s^{'},a^{'})- Q(s,a)),
$$


La principal diferencia con *generalized Q-learning* es el uso de una política que está siendo optimizada para determionar $a'$. }


Los términos $s,a,r'$ y $a'$ (de donde el nombre SARSA) debens er conocidos para actualizar la función de valor $Q$ en cada iteración.

Tanto $Q$ learning como SARSA usan las estimaciones existentes en función de valor $Q$ en cada iteración. 

El proceso bootstrapping consiste en actualizar los valores  de la función de valor $Q$ a partir des sus actuales valores con la recompensa recibida.

## Q-Learning y Open AI Gym

Open AI Gym es un conjunto de herramientas para desarrollar y comparar algoritmos de aprendizaje reofrzado.

Gym trabaja con varias librerías de deep learning incluyendoe tf.keras.

Para seguir el ejemplo de est lección instale un nuevo ambite con Gym, tensorflow y lo dempas que vaya necesitando. O ejecute los códigos desde Colaboratory (recomendado).



In [ ]:
#!pip install gym

## Ejemplo: ambiente ForzenLake-v0

Gym tiene un conjunto de ambientes (environments) en donde se peude probar algoritmos de aprendizaje reforzado (AR). En esta lección vamos a usar el ambiente ForzenLake-v0.

Este es un ambiente juguete de texto. El ambiente puede visualizarse como sigue

### Tipos de ambiente en ForzenLake-v0

ForzenLake-v0 tiene dos variantes: deslizante (slippery) y no deslizante (non-slyppery).

Como puede suponerse slippery es mas desafiante (difícil).


### Estados

ForzenLake-v0 tiene 12 estados. El estado inicial **S**, dos tipos de estados intermedios marcado como **F** (parte del lago congelada y entonces por donde puede transitar) y **H** ( que corresponde a huecod en el hielo, de donde no se puede escapar). Finalmente el estado **G** que corresponde a la meta (goal).

La recompensa es 1 si se alcanza la meta y cero en cualquier otro caso. Un episodio termina por que se alcanza la meta **G** o porque se sae en un hueco **H**.



### Acciones

ForzenLake-v0 tiene 4 posibles acciones (left, right, up, down).

Sin embargo a diferencia de Q-learnig de la lección anterior, el movimiento depende únicamente de manera parcial de la acción escogida. El movimiento depende de si el ambiente es deslizante o no. Eso es lo que lo hace más interesante.

### Observaciones

Una acción aplicada a ForzenLake-v0 retorna la observación (equivalente al estado, state), reward (recompensa), done (indicando si el episodio ha teminado) y un diccionario de información para depuración (debugging). 

Los atributos observables del ambiente los denominamos *espacio de observación*. Estos son capturado y retornado por el objeto observaci+on.

## El código

In [1]:
"""Q-Learning to solve FrozenLake-v0 problem
"""

from collections import deque
import numpy as np
import os
import time
import gym
from gym import wrappers, logger

## dieño de la clase GAgent

In [2]:
class QAgent:
    def __init__(self,
                 observation_space,
                 action_space,
                 demo=False,
                 slippery=False,
                 episodes=40000):
        """Q-Learning agent on FrozenLake-v0 environment

        Arguments:
            observation_space (tensor): state space
            action_space (tensor): action space
            demo (Bool): whether for demo or training
            slippery (Bool): 2 versions of FLv0 env
            episodes (int): number of episodes to train
        """
        
        self.action_space = action_space
        # number of columns is equal to number of actions
        col = action_space.n
        # number of rows is equal to number of states
        row = observation_space.n
        # build Q Table with row x col dims
        self.q_table = np.zeros([row, col])

        # discount factor
        self.gamma = 0.9

        # initially 90% exploration, 10% exploitation
        self.epsilon = 0.9
        # iteratively applying decay til 
        # 10% exploration/90% exploitation
        self.epsilon_min = 0.1
        self.epsilon_decay = self.epsilon_min / self.epsilon
        self.epsilon_decay = self.epsilon_decay ** \
                             (1. / float(episodes))

        # learning rate of Q-Learning
        self.learning_rate = 0.1
        
        # file where Q Table is saved on/restored fr
        if slippery:
            self.filename = '../Datos/q-frozenlake-slippery.npy'
        else:
            self.filename = '../Datos/q-frozenlake.npy'

        # demo or train mode 
        self.demo = demo
        # if demo mode, no exploration
        if demo:
            self.epsilon = 0

    def act(self, state, is_explore=False):
        """determine the next action
            if random, choose from random action space
            else use the Q Table
        Arguments:
            state (tensor): agent's current state
            is_explore (Bool): exploration mode or not
        Return:
            action (tensor): action that the agent
                must execute
        """
        # 0 - left, 1 - Down, 2 - Right, 3 - Up
        if is_explore or np.random.rand() < self.epsilon:
            # explore - do random action
            return self.action_space.sample()

        # exploit - choose action with max Q-value
        action = np.argmax(self.q_table[state])
        return action


    def update_q_table(self, state, action, reward, next_state):
        """TD(0) learning (generalized Q-Learning) with learning rate
        Arguments:
            state (tensor): environment state
            action (tensor): action executed by the agent for
                the given state
            reward (float): reward received by the agent for
                executing the action
            next_state (tensor): the environment next state
        """
        # Q(s, a) += 
        # alpha * (reward + gamma * max_a' Q(s', a') - Q(s, a))
        q_value = self.gamma * np.amax(self.q_table[next_state])
        q_value += reward
        q_value -= self.q_table[state, action]
        q_value *= self.learning_rate
        q_value += self.q_table[state, action]
        self.q_table[state, action] = q_value


    def print_q_table(self):
        """dump Q Table"""
        print(self.q_table)
        print("Epsilon : ", self.epsilon)


    def save_q_table(self):
        """save trained Q Table"""
        np.save(self.filename, self.q_table)


    def load_q_table(self):
        """load trained Q Table"""
        self.q_table = np.load(self.filename)


    def update_epsilon(self):
        """adjust epsilon"""
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

## Configura 

[gym: Enviroments, Wrappers and Monitors](https://hub.packtpub.com/openai-gym-environments-wrappers-and-monitors-tutorial/)

In [8]:
# instantiate a gym environment (FrozenLake-v0)
# parameters
env_id ='FrozenLake-v0'
# slippery or not
slippery = True # or False
# time to delay
delay = None  # or a time in secoids to delay: delay = 1
# debug dir
outdir = "../Datos/q-learning-%s" %env_id
# mode demo?
demo = True # or True
# explorar?
explore = True

# instantiate a gym environment
env = gym.make(env_id)
env = wrappers.Monitor(env, directory=outdir, force=True)
env.seed(0)

# configure the trainig
if not slippery:
    env.is_slippery = False

if delay is not None:
    delay = args.delay 
else: 
    delay = 0

# number of times the Goal state is reached
wins = 0
# number of episodes to train
episodes = 4000

# instantiate a Q Learning agent
agent = QAgent(env.observation_space,
               env.action_space,
               demo=demo,
               slippery=slippery,
               episodes=episodes)

if demo:
    agent.load_q_table()

## Corre los episodios de entrenamiento

In [ ]:
# loop for the specified number of episode
for episode in range(episodes):
    state = env.reset()
    done = False
    while not done:
        # determine the agent's action given state
        action = agent.act(state, is_explore=explore)
        # get observable data
        next_state, reward, done, _ = env.step(action)
        # clear the screen before rendering the environment
        os.system('clear')
        # render the environment for human debugging
        env.render()
        # training of Q Table
        if done:
            # update exploration-exploitation ratio
            # reward > 0 only when Goal is reached
            # otherwise, it is a Hole
            if reward > 0:
                wins += 1

        if not demo:
            agent.update_q_table(state,
                                action, 
                                 reward, 
                                 next_state)
            agent.update_epsilon()

        state = next_state
        percent_wins = 100.0 * wins / (episode + 1)
        print("-------%0.2f%% Goals in %d Episodes---------"
                % (percent_wins, episode))
        if done:
            time.sleep(5 * delay)
        else:
            time.sleep(delay)


print("Episodes: ", episode)
print("Goals/Holes: %d/%d" % (wins, episode - wins))

agent.print_q_table()
if not args.demo and not args.explore:
    agent.save_q_table()

# close the env and write monitor result info to disk
env.close() 

  (Down)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 0 Episodes---------
  (Right)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 0 Episodes---------
  (Right)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 0 Episodes---------
  (Left)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 0 Episodes---------
  (Down)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 0 Episodes---------
  (Down)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 0 Episodes---------
  (Up)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 1 Episodes---------
  (Down)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 1 Episodes---------
  (Left)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 2 Episodes---------
  (Left)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 2 Episodes---------
  (Left)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 2 Episodes---------
  (Left)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 2 Episodes---------
  (Down)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 2 Episodes---------
  (Left)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 2 Episodes---------
  (Rig